# Client 동기화

In [169]:
import kfp
import requests

USERNAME = "user@example.com"
PASSWORD = "12341234"
NAMESPACE = "kubeflow-user-example-com"
HOST = "http://172.17.0.12:8080" # istio-ingressgateway pod ip:port

session = requests.Session()
response = session.get(HOST)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {"login": "user@example.com", "password": "12341234"}
session.post(response.url, headers=headers, data=data)
session_cookie = session.cookies.get_dict()["authservice_session"]

client = kfp.Client(
    host=f"{HOST}/pipeline",
    namespace=f"{NAMESPACE}",
    cookies=f"authservice_session={session_cookie}",
)
list_pipelines = client.list_pipelines()

for i in range(list_pipelines.total_size):
    print(list_pipelines.pipelines[i].name)
print(response.url)
print(session_cookie)

[Demo] XGBoost - Iterative model training
[Demo] TFX - Taxi tip prediction model trainer
[Tutorial] Data passing in python components
[Tutorial] DSL - Control structures
kf_iris_pipeline
http://172.17.0.12:8080/dex/auth/local/login?back=&state=d6u5pi5snk7gz634bwdikbkvm
MTY3MjE0NTE0NXxOd3dBTkZSVU1reEdUa3hCVXpaVFVVZEJRVXcyVDBaRk5UWktOalZUV0VkWFVEWkhUVmszUlU5SU56ZFRWbFZOV2xnMlRrWklSRUU9fO8iggyT7WJR2__YsjhgPGBlGnYqS37qZE9cuWd41dJo


In [170]:
client

# Experiment 정보 가져오기

In [199]:
experiment_info = client.get_experiment(experiment_name="mnist-pipeline",namespace=NAMESPACE)

In [200]:
experiment_id = experiment_info.id

# Experiment 삭제하기

In [201]:
client.delete_experiment(experiment_id=experiment_id)

{}

# Runs 삭제하기

In [204]:
for i in range(client.list_runs().total_size):
    run_id = client.list_runs().runs[i].id
    client.runs.delete_run(id=run_id)
    print(run_id)

d346b4a9-130d-4338-8e54-ea4f7bfdec4e
b831af3a-ea79-4514-8b17-df9462fedef9
200f46bf-9f75-412a-ad75-1563b522fa34
9040cc7a-7569-4b43-ad7e-141cd07f80e8


{}

## Pipeline Upload

In [196]:
pipeline_package_path='kfp_practice/multiply/multiply-pipeline.yaml'
pipeline_name='multiply-pipeline'
client.upload_pipeline(pipeline_package_path=pipeline_package_path,pipeline_name=pipeline_name)

{'created_at': datetime.datetime(2022, 12, 27, 14, 21, 13, tzinfo=tzlocal()),
 'default_version': {'code_source_url': None,
                     'created_at': datetime.datetime(2022, 12, 27, 14, 21, 13, tzinfo=tzlocal()),
                     'id': 'fd054cdd-2653-4457-9347-5986523b6106',
                     'name': 'multiply-pipeline',
                     'package_url': None,
                     'parameters': None,
                     'resource_references': [{'key': {'id': 'fd054cdd-2653-4457-9347-5986523b6106',
                                                      'type': 'PIPELINE'},
                                              'name': None,
                                              'relationship': 'OWNER'}]},
 'description': None,
 'error': None,
 'id': 'fd054cdd-2653-4457-9347-5986523b6106',
 'name': 'multiply-pipeline',
 'parameters': None,
 'resource_references': None,
 'url': None}

# Experiment 생성

In [197]:
experiment = client.create_experiment(name=pipeline_name,description=pipeline_name,namespace=NAMESPACE)

## Run 생성

In [198]:
client.run_pipeline(experiment_id=experiment.id,job_name=pipeline_name,pipeline_package_path=pipeline_package_path)

{'created_at': datetime.datetime(2022, 12, 27, 14, 21, 19, tzinfo=tzlocal()),
 'description': None,
 'error': None,
 'finished_at': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
 'id': '9040cc7a-7569-4b43-ad7e-141cd07f80e8',
 'metrics': None,
 'name': 'multiply-pipeline',
 'pipeline_spec': {'parameters': None,
                   'pipeline_id': None,
                   'pipeline_manifest': None,
                   'pipeline_name': None,
                   'workflow_manifest': '{"apiVersion": '
                                        '"argoproj.io/v1alpha1", "kind": '
                                        '"Workflow", "metadata": '
                                        '{"generateName": '
                                        '"my-multiply-component-pipeline-", '
                                        '"annotations": '
                                        '{"pipelines.kubeflow.org/kfp_sdk_version": '
                                        '"1.6.3", '
                 